## Building basic likelihoods

In [1]:
import sys
import os
import glob

import tqdm

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

%matplotlib inline

In [3]:
import jax
jax.config.update('jax_enable_x64', True)

import jax.random
import jax.numpy as jnp

In [4]:
import discovery as ds
import discovery.models.nanograv as ds_nanograv
import discovery.samplers.numpyro as ds_numpyro

Read nanograv pulsars

In [5]:
allpsrs = [ds.Pulsar.read_feather(psrfile) for psrfile in sorted(glob.glob('../data/*-[JB]*.feather'))]

In [6]:
psr = allpsrs[0]

### Single pulsar likelihood

#### Measurement noise only, no backends

In [7]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement_simple(psr)])

What are the active parameters?

In [8]:
m.logL.params

['B1855+09_efac', 'B1855+09_log10_t2equad']

Sample random values from their priors

In [11]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_efac': 0.9884360248508917,
 'B1855+09_log10_t2equad': -6.3945137607653}

Evaluate the likelihood

In [12]:
m.logL(p0)

Array(97726.97619403, dtype=float64)

Try compiled version, grad

In [13]:
jax.jit(m.logL)(p0)

Array(97726.97619403, dtype=float64)

In [14]:
jax.grad(m.logL)(p0)

{'B1855+09_efac': Array(6174.79766028, dtype=float64, weak_type=True),
 'B1855+09_log10_t2equad': Array(4569.05261176, dtype=float64, weak_type=True)}

#### Measurement noise only, nanograv backends, free parameters

In [15]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr)])

In [16]:
m.logL.params

['B1855+09_430_ASP_efac',
 'B1855+09_430_PUPPI_efac',
 'B1855+09_L-wide_ASP_efac',
 'B1855+09_L-wide_PUPPI_efac',
 'B1855+09_430_ASP_log10_t2equad',
 'B1855+09_430_PUPPI_log10_t2equad',
 'B1855+09_L-wide_ASP_log10_t2equad',
 'B1855+09_L-wide_PUPPI_log10_t2equad']

In [20]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_430_ASP_efac': 0.9912537195541836,
 'B1855+09_430_PUPPI_efac': 0.9010861604720153,
 'B1855+09_L-wide_ASP_efac': 0.9216015906296419,
 'B1855+09_L-wide_PUPPI_efac': 0.994668046378104,
 'B1855+09_430_ASP_log10_t2equad': -6.4175268942552615,
 'B1855+09_430_PUPPI_log10_t2equad': -8.324684529665847,
 'B1855+09_L-wide_ASP_log10_t2equad': -5.323020662736043,
 'B1855+09_L-wide_PUPPI_log10_t2equad': -6.3524754567384845}

In [21]:
m.logL(p0)

Array(98528.0782194, dtype=float64)

#### Measurement noise only, nanograv backends, parameters from noisedict

In [22]:
psr.noisedict

{'B1855+09_430_ASP_efac': 1.115935306813982,
 'B1855+09_430_ASP_log10_t2equad': -7.564164330699591,
 'B1855+09_430_PUPPI_efac': 1.000049037085653,
 'B1855+09_430_PUPPI_log10_t2equad': -6.572540211467256,
 'B1855+09_L-wide_ASP_efac': 1.043114017270374,
 'B1855+09_L-wide_ASP_log10_t2equad': -6.517929916655293,
 'B1855+09_L-wide_PUPPI_efac': 1.1118432332882,
 'B1855+09_L-wide_PUPPI_log10_t2equad': -7.755603780476984,
 'B1855+09_430_ASP_log10_ecorr': -6.798122106550257,
 'B1855+09_430_PUPPI_log10_ecorr': -5.6989064141929715,
 'B1855+09_L-wide_ASP_log10_ecorr': -6.120457109433745,
 'B1855+09_L-wide_PUPPI_log10_ecorr': -6.641667916624413,
 'B1855+09_red_noise_log10_A': -13.940953818371378,
 'B1855+09_red_noise_gamma': -3.68432133461766}

In [23]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict)])

In [24]:
m.logL.params

[]

In [25]:
m.logL(ds.sample_uniform(m.logL.params))

np.float64(96686.81011373615)

#### Add ECORR noise (GP), free params

In [26]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr),
                         ds.makegp_ecorr(psr)])

In [27]:
m.logL.params

['B1855+09_430_ASP_efac',
 'B1855+09_430_ASP_log10_ecorr',
 'B1855+09_430_ASP_log10_t2equad',
 'B1855+09_430_PUPPI_efac',
 'B1855+09_430_PUPPI_log10_ecorr',
 'B1855+09_430_PUPPI_log10_t2equad',
 'B1855+09_L-wide_ASP_efac',
 'B1855+09_L-wide_ASP_log10_ecorr',
 'B1855+09_L-wide_ASP_log10_t2equad',
 'B1855+09_L-wide_PUPPI_efac',
 'B1855+09_L-wide_PUPPI_log10_ecorr',
 'B1855+09_L-wide_PUPPI_log10_t2equad']

In [34]:
p0 = ds.sample_uniform(m.logL.params);
p0

{'B1855+09_430_ASP_efac': 0.9784963941495879,
 'B1855+09_430_ASP_log10_ecorr': -5.289010348880711,
 'B1855+09_430_ASP_log10_t2equad': -8.208858428390645,
 'B1855+09_430_PUPPI_efac': 1.023131776484832,
 'B1855+09_430_PUPPI_log10_ecorr': -8.048103980290096,
 'B1855+09_430_PUPPI_log10_t2equad': -7.082921616765829,
 'B1855+09_L-wide_ASP_efac': 1.0960023227616964,
 'B1855+09_L-wide_ASP_log10_ecorr': -6.642180707524991,
 'B1855+09_L-wide_ASP_log10_t2equad': -5.602892196176345,
 'B1855+09_L-wide_PUPPI_efac': 0.983861693759383,
 'B1855+09_L-wide_PUPPI_log10_ecorr': -5.280146731477677,
 'B1855+09_L-wide_PUPPI_log10_t2equad': -7.960038365259047}

In [35]:
m.logL(p0)

Array(99649.99202655, dtype=float64)

#### Add ECORR noise (GP), noisedict params

In [36]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict)])

In [37]:
m.logL.params

[]

In [38]:
m.logL(ds.sample_uniform(m.logL.params))

np.float64(100267.11920382788)

#### Add timing model

In [39]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True)])

In [40]:
m.logL.params

[]

In [41]:
m.logL(ds.sample_uniform(m.logL.params))

np.float64(90998.4965539236)

#### Add red noise (powerlaw)

In [42]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True),
                         ds.makegp_fourier(psr, ds.powerlaw, components=30, name='rednoise')])

In [43]:
m.logL.params

['B1855+09_rednoise_gamma', 'B1855+09_rednoise_log10_A']

In [44]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_rednoise_gamma': 1.8661431518556293,
 'B1855+09_rednoise_log10_A': -15.767702243848015}

In [45]:
m.logL(p0)

Array(90998.53181259, dtype=float64)

#### Add red noise (powerlaw, fixed gamma)

In [46]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True),
                         ds.makegp_fourier(psr, ds.partial(ds.powerlaw, gamma=4.33), components=30, name='rednoise')])

In [47]:
m.logL.params

['B1855+09_rednoise_log10_A']

In [49]:
p0 = ds.sample_uniform(m.logL.params);
p0

{'B1855+09_rednoise_log10_A': -14.56138230938988}

In [50]:
m.logL(p0)

Array(91087.3353244, dtype=float64)

#### Add red noise (free spectrum)

In [51]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True),
                         ds.makegp_fourier(psr, ds.freespectrum, components=30, name='rednoise')])

In [52]:
m.logL.params

['B1855+09_rednoise_log10_rho(30)']

In [53]:
p0 = {'B1855+09_rednoise_log10_rho(30)': 1e-2 * np.random.randn(30)}; p0

{'B1855+09_rednoise_log10_rho(30)': array([-0.0022863 ,  0.00526641, -0.00772843,  0.00358156, -0.01388954,
        -0.01384554, -0.00425084,  0.00341102, -0.00057387,  0.00823223,
         0.01291044, -0.00685494, -0.0079893 ,  0.01707681, -0.01717273,
        -0.01478698, -0.00234845,  0.01623931,  0.0038275 , -0.01032555,
         0.00591531, -0.00595341,  0.00835566,  0.02387495,  0.02271765,
        -0.00330971,  0.00730778, -0.01454399, -0.0046486 ,  0.00212594])}

In [54]:
m.logL(p0)

Array(90205.36824127, dtype=float64)

### Multiple pulsars

In [55]:
psrs = allpsrs[:3]

#### Combined likelihood

In [56]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=30, name='rednoise')])
                        for psr in psrs])

In [57]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A']

In [58]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_rednoise_gamma': 1.3138772930930416,
 'B1855+09_rednoise_log10_A': -17.6973773044854,
 'B1937+21_rednoise_gamma': 5.138854166981884,
 'B1937+21_rednoise_log10_A': -18.37138668421632,
 'B1953+29_rednoise_gamma': 2.740277693849858,
 'B1953+29_rednoise_log10_A': -15.896408518581373}

In [60]:
m.logL(p0)

Array(428689.00376984, dtype=float64)

#### Add common noise

Indicating parameters under common shares them among pulsars

In [69]:
T = ds.getspan(psrs)

In [70]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=30, T=T, name='rednoise'),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=14, T=T, name='crn',
                                                               common=['crn_log10_A', 'crn_gamma'])])
                        for psr in psrs])

In [71]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'crn_gamma',
 'crn_log10_A']

In [67]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_rednoise_gamma': 4.393246413466512,
 'B1855+09_rednoise_log10_A': -13.40258009298283,
 'B1937+21_rednoise_gamma': 6.962929660446664,
 'B1937+21_rednoise_log10_A': -14.279437020259682,
 'B1953+29_rednoise_gamma': 6.629185927710269,
 'B1953+29_rednoise_log10_A': -16.940946338152244,
 'crn_gamma': 5.554819975636826,
 'crn_log10_A': -16.898578025214864}

In [74]:
m.logL(p0)

Array(473834.57989403, dtype=float64)

#### Parallelize red components

Coordinated timespan is required

In [44]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True)]) for psr in psrs],
                       commongp = [ds.makecommongp_fourier(psrs, ds.powerlaw, components=30, T=T, name='rednoise'),
                                   ds.makecommongp_fourier(psrs, ds.powerlaw, components=14, T=T, name='crn',
                                                           common=['crn_log10_A', 'crn_gamma'])])

In [45]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'crn_gamma',
 'crn_log10_A']

In [48]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_rednoise_gamma': 6.2016324259132976,
 'B1855+09_rednoise_log10_A': -16.51182106267872,
 'B1937+21_rednoise_gamma': 5.417321744296163,
 'B1937+21_rednoise_log10_A': -14.81401491518855,
 'B1953+29_rednoise_gamma': 5.802346787367874,
 'B1953+29_rednoise_log10_A': -18.89307111376218,
 'crn_gamma': 6.518869741837323,
 'crn_log10_A': -11.52173724907908}

In [49]:
m.logL(p0)

Array(473520.64257291, dtype=float64)

#### Reuse Fourier vectors

`ds.makepowerlaw_crn` yields the sum of two powerlaws, with possibly different number of components.

In [75]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True)]) for psr in psrs],
                       commongp = ds.makecommongp_fourier(psrs, ds.makepowerlaw_crn(components=14), components=30, T=T, name='rednoise',
                                                          common=['crn_log10_A', 'crn_gamma']))

In [76]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'crn_gamma',
 'crn_log10_A']

In [78]:
p0

{'B1855+09_rednoise_gamma': 4.393246413466512,
 'B1855+09_rednoise_log10_A': -13.40258009298283,
 'B1937+21_rednoise_gamma': 6.962929660446664,
 'B1937+21_rednoise_log10_A': -14.279437020259682,
 'B1953+29_rednoise_gamma': 6.629185927710269,
 'B1953+29_rednoise_log10_A': -16.940946338152244,
 'crn_gamma': 5.554819975636826,
 'crn_log10_A': -16.898578025214864}

In [77]:
m.logL(p0)

Array(473834.57989403, dtype=float64)

#### Add global spatially correlated process

Note `ds.makeglobalgp_fourier` requires the ORF, but not the `common` specification, which is automatic.

In [79]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True)]) for psr in psrs],
                       commongp = ds.makecommongp_fourier(psrs, ds.powerlaw, components=30, T=T, name='rednoise'),
                       globalgp = ds.makeglobalgp_fourier(psrs, ds.powerlaw, ds.hd_orf, components=14, T=T, name='gw'))

In [80]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'gw_gamma',
 'gw_log10_A']

In [81]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_rednoise_gamma': 0.3053862046782503,
 'B1855+09_rednoise_log10_A': -17.37451609504897,
 'B1937+21_rednoise_gamma': 6.829462349490865,
 'B1937+21_rednoise_log10_A': -16.901071401725815,
 'B1953+29_rednoise_gamma': 3.199765814340998,
 'B1953+29_rednoise_log10_A': -15.797254502360095,
 'gw_gamma': 6.48295163543074,
 'gw_log10_A': -11.832525971036855}

In [82]:
m.logL(p0)

Array(473579.69075438, dtype=float64)

#### Another way of doing this (useful if variable GPs differ among pulsars)

In [83]:
m = ds.GlobalLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=30, name='rednoise')]) for psr in psrs],
                        globalgp = ds.makeglobalgp_fourier(psrs, ds.powerlaw, ds.hd_orf, components=14, T=T, name='gw'))

In [84]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'gw_gamma',
 'gw_log10_A']

In [85]:
p0

{'B1855+09_rednoise_gamma': 0.3053862046782503,
 'B1855+09_rednoise_log10_A': -17.37451609504897,
 'B1937+21_rednoise_gamma': 6.829462349490865,
 'B1937+21_rednoise_log10_A': -16.901071401725815,
 'B1953+29_rednoise_gamma': 3.199765814340998,
 'B1953+29_rednoise_log10_A': -15.797254502360095,
 'gw_gamma': 6.48295163543074,
 'gw_log10_A': -11.832525971036855}

In [86]:
m.logL(p0)

Array(473579.69075476, dtype=float64)